In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Masking
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print('Modulos Importados')

Modulos Importados


In [10]:
df = pd.read_csv("dataset_RNN.csv")
df.head()

,userId,movieId,timestamp,temporal_1,temporal_2,temporal_3,rating
0,1,541,943227521,-1.684309,-0.800246,1.356669,5.0
1,1,166,943228442,-1.679724,-0.797711,1.357109,5.0
2,1,1784,943228545,-0.531675,1.601560,2.422522,1.0
3,1,1944,943231120,-1.256493,2.702224,-1.212740,2.0
4,1,1208,943231192,-2.109507,0.907206,-2.011031,5.0


In [11]:
df = df.sort_values(by=["userId", "timestamp"]).reset_index(drop=True)

In [12]:
sequence_data = []
target_data = []

for uid, group in df.groupby("userId"):
    features = group[["temporal_1", "temporal_2", "temporal_3"]].values
    ratings = group["rating"].values
    
    if len(features) > 1:
        # Input = todas menos la última, Target = todas menos la primera
        sequence_data.append(features[:-1])
        target_data.append(ratings[1:])

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(sequence_data, padding='post', dtype='float32', maxlen=100)
y = pad_sequences(target_data, padding='post', dtype='float32', maxlen=100)

In [14]:
# Expandir la dimensión final para que sea compatible con output del modelo
y = np.expand_dims(y, axis=-1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

c:\Python312\Lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_1 (Masking)             │ (None, 100, 3)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 100, 64)        │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100, 1)         │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,417 (17.25 KB)

 Trainable params: 4,417 (17.25 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_val, y_val))

Epoch 1/5
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - loss: 2.4590 - val_loss: 0.9759
Epoch 2/5
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - loss: 0.9630 - val_loss: 0.9452
Epoch 3/5
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - loss: 0.9513 - val_loss: 0.9442
Epoch 4/5
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - loss: 0.9485 - val_loss: 0.9412
Epoch 5/5
1237/1237 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - loss: 0.9512 - val_loss: 0.9510


In [17]:
predictions = model.predict(X_val)

1237/1237 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step


In [18]:
mask = (y_val != 0)
rmse = np.sqrt(mean_squared_error(y_val[mask], predictions[mask]))
print(f"RMSE en test_set_1: {rmse:.4f}")


RMSE en test_set_1: 0.9745
